# Passing exercises 

In [2]:
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import * 



## Exercise 0

In [2]:
!ping -6 facebook.com -c 4

PING facebook.com(edge-star-mini6-shv-02-iad3.facebook.com (2a03:2880:f103:181:face:b00c:0:25de)) 56 data bytes
64 bytes from edge-star-mini6-shv-02-iad3.facebook.com (2a03:2880:f103:181:face:b00c:0:25de): icmp_seq=1 ttl=49 time=0.531 ms
64 bytes from edge-star-mini6-shv-02-iad3.facebook.com (2a03:2880:f103:181:face:b00c:0:25de): icmp_seq=2 ttl=49 time=0.560 ms
64 bytes from edge-star-mini6-shv-02-iad3.facebook.com (2a03:2880:f103:181:face:b00c:0:25de): icmp_seq=3 ttl=49 time=0.522 ms
64 bytes from edge-star-mini6-shv-02-iad3.facebook.com (2a03:2880:f103:181:face:b00c:0:25de): icmp_seq=4 ttl=49 time=0.572 ms

--- facebook.com ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3021ms
rtt min/avg/max/mdev = 0.522/0.546/0.572/0.020 ms


In [3]:
host = "ipv6.google.com"

In [6]:
i=IPv6(dst=host)
q=ICMPv6EchoRequest()
p=(i/q)
ans = sr1(p,timeout=3, verbose=1)

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


In [7]:
ans.show()


###[ IPv6 ]### 
  version   = 6
  tc        = 0
  fl        = 0
  plen      = 8
  nh        = ICMPv6
  hlim      = 99
  src       = 2607:f8b0:4004:c1b::64
  dst       = 2600:1f18:432d:43a1:6454:f46:6121:cfa3
###[ ICMPv6 Echo Reply ]### 
     type      = Echo Reply
     code      = 0
     cksum     = 0xa33b
     id        = 0x0
     seq       = 0x0
     data      = ''



## Exercise 1

In [62]:
!ping -6 2604:5f00:ffff:fe00::101:ca75 -c 3

PING 2604:5f00:ffff:fe00::101:ca75(2604:5f00:ffff:fe00::101:ca75) 56 data bytes
From 2604:5f00:ffff:ffff::2 icmp_seq=1 Destination unreachable: Unknown code 6
From 2604:5f00:ffff:ffff::2 icmp_seq=2 Destination unreachable: Unknown code 6
From 2604:5f00:ffff:ffff::2 icmp_seq=3 Destination unreachable: Unknown code 6

--- 2604:5f00:ffff:fe00::101:ca75 ping statistics ---
3 packets transmitted, 0 received, +3 errors, 100% packet loss, time 2004ms



In [68]:
ans = sr1(IPv6(dst="2604:5f00:ffff:fe00::101:ca75")/ICMPv6EchoRequest(),timeout=3, verbose=1)
ans.show()

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
###[ IPv6 ]### 
  version   = 6
  tc        = 0
  fl        = 0
  plen      = 56
  nh        = ICMPv6
  hlim      = 45
  src       = 2604:5f00:ffff:ffff::2
  dst       = 2600:1f18:432d:43a1:6454:f46:6121:cfa3
###[ ICMPv6 Destination Unreachable ]### 
     type      = Destination unreachable
     code      = 6
     cksum     = 0x6f43
     length    = 0
     unused    = 0x0
###[ IPv6 in ICMPv6 ]### 
        version   = 6
        tc        = 0
        fl        = 0
        plen      = 8
        nh        = ICMPv6
        hlim      = 49
        src       = 2600:1f18:432d:43a1:6454:f46:6121:cfa3
        dst       = 2604:5f00:ffff:fe00::101:ca75
###[ ICMPv6 Echo Request ]### 
           type      = Echo Request
           code      = 0
           cksum     = 0xc0fa
           id        = 0x0
           seq       = 0x0
           data      = ''



**Note:** The IPv6 address `2604:5f00:ffff:fe00::101:ca75` is unreachable, so I am going to use one of my hosts with IPv6.

My IPv6 host: `2600:1f18:432d:43a1:6454:f46:6121:cfa3`


### Send my lastname using ICMPv6

In [60]:
#send last name as payload 
#using one of my hosts as destination
host_ipv6="2600:1f18:432d:43a1:6454:f46:6121:cfa3"
payload = b"Barrios"

In [10]:
type(payload)

bytes

In [13]:
send(IPv6(dst=host_ipv6) / ICMPv6EchoRequest() /payload , iface='eth0')


.
Sent 1 packets.


#### Sniffer host output - Lastname
```bash
>>> pkts=sniff(iface="eth0",lfilter = lambda x: x.haslayer(IPv6))
>>> pkts.show()
0000 Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 0a:16:23:17:11:01
0001 Ether / IPv6 / ICMPv6 Echo Request (id: 0x0 seq: 0x0)
0002 Ether / IPv6 / ICMPv6ND_RA / ICMPv6NDOptSrcLLAddr / ICMPv6 Neighbor Discovery Option - Prefix Information 2600:1f18:432d:43a1::/64 On-link 1 Autonomous Address 0 Router Address 0
>>> pkts[1]
<Ether  dst=ff:ff:ff:ff:ff:ff src=0a:16:23:17:11:01 type=IPv6 |<IPv6  version=6 tc=0 fl=0 plen=15 nh=ICMPv6 hlim=64 src=2600:1f18:432d:43a1:6454:f46:6121:cfa3 dst=2600:1f18:432d:43a1:6454:f46:6121:cfa3 |<ICMPv6EchoRequesttype=Echo Request code=0 cksum=0xde6 id=0x0 seq=0x0 data='Barrios' |>>>
```

### Send long string using ICMPv6

In [54]:
long_string="Internet Protocol version 6 (IPv6) is the most recent version of the Internet Protocol (IP), the communications protocol that provides an identification and location system for computers on" \
            "networks and routes traffic across the Internet. IPv6 was developed by the Internet Engineering Task Force (IETF)" \
            "to deal with the long-anticipated problem of IPv4 address exhaustion, and is intended to replace IPv4. In December 1998," \
            "IPv6 became a Draft Standard for the IETF, which subsequently ratified it as an Internet Standard on 14 July 2017.Devices on" \
            "the Internet are assigned a unique IP address for identification and location definition. With the rapid growth of the Internet" \
            "after commercialization in the 1990s, it became evident that far more addresses would be needed to connect devices than the IPv4" \
            "address space had available.By 1998, the IETF had formalized the successor protocol. IPv6 uses 128-bit addresses," \
            "theoretically allowing 2128, or approximately 3.4×1038 total addresses. The actual number is slightly smaller, as multiple" \
            "ranges are reserved for special use or completely excluded from use. The two protocols are not designed to be interoperable," \
            "and thus direct communication between them is impossible, complicating the move to IPv6. However, several transition mechanisms" \
            "have been devised to rectify this. IPv6 provides other technical benefits in addition to a larger addressing space. In particular," \
            "it permits hierarchical address allocation methods that facilitate route aggregation across the Internet, and thus limit the expansion of routing tables." \
            "The use of multicast addressing is expanded and simplified, and provides additional optimization for the delivery of services. Device mobility, security," \
            "and configuration aspects have been considered in the design of the protocol." 
           

In [55]:
bytes_long = long_string.encode()

In [71]:
print("String length:",len(bytes_long))

String length: 1801


In [57]:
#send long string
send(IPv6(dst=host_ipv6) / ICMPv6EchoRequest() /bytes_long , iface='eth0')


.
Sent 1 packets.


#### Sniffer host output - Long String
```bash
>>> pkts.show()
0000 Ether / fe80::816:23ff:fe17:1101 > ff02::16 (0) / IPv6ExtHdrHopByHop / ICMPv6MLReport2
0001 Ether / fe80::816:23ff:fe17:1101 > ff02::16 (0) / IPv6ExtHdrHopByHop / ICMPv6MLReport2
0002 Ether / IPv6 / ICMPv6ND_NS / ICMPv6 Neighbor Discovery Option - Source Link-Layer Address 0a:16:23:17:11:01
0003 Ether / IPv6 / ICMPv6 Echo Request (id: 0x0 seq: 0x0)
0004 Ether / IPv6 / ICMPv6ND_RA / ICMPv6NDOptSrcLLAddr / ICMPv6 Neighbor Discovery Option - Prefix Information 2600:1f18:432d:43a1::/64 On-link 1 Autonomous Address 0 Router Address 0
>>> pkts[3].show()
###[ Ethernet ]###
  dst       = ff:ff:ff:ff:ff:ff
  src       = 0a:16:23:17:11:01
  type      = IPv6
###[ IPv6 ]###
     version   = 6
     tc        = 0
     fl        = 0
     plen      = 1809
     nh        = ICMPv6
     hlim      = 64
     src       = 2600:1f18:432d:43a1:6454:f46:6121:cfa3
     dst       = 2600:1f18:432d:43a1:6454:f46:6121:cfa3
###[ ICMPv6 Echo Request ]###
        type      = Echo Request
        code      = 0
        cksum     = 0x39c9
        id        = 0x0
        seq       = 0x0
        data      = 'Internet Protocol version 6 (IPv6) is the most recent version of the Internet Protocol (IP), the communications protocol that provides an identification and location system for computers onnetworks and routes traffic across the Internet. IPv6 was developed by the Internet Engineering Task Force (IETF)to deal with the long-anticipated problem of IPv4 address exhaustion, and is intended to replace IPv4. In December 1998,IPv6 became a Draft Standard for the IETF, which subsequently ratified it as an Internet Standard on 14 July 2017.Devices onthe Internet are assigned a unique IP address for identification and location definition. With the rapid growth of the Internetafter commercialization in the 1990s, it became evident that far more addresses would be needed to connect devices than the IPv4address space had available.By 1998, the IETF had formalized the successor protocol. IPv6 uses 128-bit addresses,theoretically allowing 2128, or approximately 3.4×1038 total addresses. The actual number is slightly smaller, as multipleranges are reserved for special use or completely excluded from use. The two protocols are not designed to be interoperable,and thus direct communication between them is impossible, complicating the move to IPv6. However, several transition mechanismshave been devised to rectify this. IPv6 provides other technical benefits in addition to a larger addressing space. In particular,it permits hierarchical address allocation methods that facilitate route aggregation across the Internet, and thus limit the expansion of routing tables.The use of multicast addressing is expanded and simplified, and provides additional optimization for the delivery of services. Device mobility, security,and configuration aspects have been considered in the design of the protocol.'
>>> len(pkts[3])
1863
>>> pkts[3].data
b'Internet Protocol version 6 (IPv6) is the most recent version of the Internet Protocol (IP), the communications protocol that provides an identification and location system for computers onnetworks and routes traffic across the Internet. IPv6 was developed by the Internet Engineering Task Force (IETF)to deal with the long-anticipated problem of IPv4 address exhaustion, and is intended to replace IPv4. In December 1998,IPv6 became a Draft Standard for the IETF, which subsequently ratified it as an Internet Standard on 14 July 2017.Devices onthe Internet are assigned a unique IP address for identification and location definition. With the rapid growth of the Internetafter commercialization in the 1990s, it became evident that far more addresses would be needed to connect devices than the IPv4address space had available.By 1998, the IETF had formalized the successor protocol. IPv6 uses 128-bit addresses,theoretically allowing 2128, or approximately 3.4\xc3\x971038 total addresses. The actual number is slightly smaller, as multipleranges are reserved for special use or completely excluded from use. The two protocols are not designed to be interoperable,and thus direct communication between them is impossible, complicating the move to IPv6. However, several transition mechanismshave been devised to rectify this. IPv6 provides other technical benefits in addition to a larger addressing space. In particular,it permits hierarchical address allocation methods that facilitate route aggregation across the Internet, and thus limit the expansion of routing tables.The use of multicast addressing is expanded and simplified, and provides additional optimization for the delivery of services. Device mobility, security,and configuration aspects have been considered in the design of the protocol.'
>>> len(pkts[3].data)
1801
```

### Testing my DNS using IPv6

In [74]:
#testing my DNS over IPv6
send(IPv6(dst="ipv6.waybarrios.com") / ICMPv6EchoRequest() /b'Testing DNS using IPv6' , iface='eth0')

.
Sent 1 packets.


#### Sniffer host output - DNS IPv6
```bash
>>> pkts=sniff(iface="eth0",lfilter = lambda x: x.haslayer(IPv6))
>>> pkts.show()
0000 Ether / IPv6 / ICMPv6 Echo Request (id: 0x0 seq: 0x0)
0001 Ether / IPv6 / ICMPv6 Echo Reply (id: 0x0 seq: 0x0)
0002 Ether / IPv6 / ICMPv6ND_RA / ICMPv6NDOptSrcLLAddr / ICMPv6 Neighbor Discovery Option - Prefix Information 2600:1f18:432d:43a1::/64 On-link 1 Autonomous Address 0 Router Address 0
>>> pkts[0].show()
###[ Ethernet ]###
  dst       = 0a:54:50:8c:3c:29
  src       = 0a:16:23:17:11:01
  type      = IPv6
###[ IPv6 ]###
     version   = 6
     tc        = 0
     fl        = 0
     plen      = 30
     nh        = ICMPv6
     hlim      = 64
     src       = 2600:1f18:432d:43a1:6454:f46:6121:cfa3
     dst       = 2606:4700:3031::6815:427e
###[ ICMPv6 Echo Request ]###
        type      = Echo Request
        code      = 0
        cksum     = 0x9235
        id        = 0x0
        seq       = 0x0
        data      = 'Testing DNS using IPv6'
```

## Exercise 2

In [82]:
!traceroute6 google.com

traceroute to google.com (2607:f8b0:4004:c06::66), 30 hops max, 80 byte packets
 1  2620:107:4000:4009:8000:0:6441:1f80 (2620:107:4000:4009:8000:0:6441:1f80)  7.678 ms * 2620:107:4000:4009:8000:0:6441:4a80 (2620:107:4000:4009:8000:0:6441:4a80)  1.791 ms
 2  * 2620:107:4000:400d:8000:0:6442:984 (2620:107:4000:400d:8000:0:6442:984)  7.549 ms 2620:107:4000:400d:8000:0:6442:d8c (2620:107:4000:400d:8000:0:6442:d8c)  8.186 ms
 3  2620:107:4000:c5c2::f3fd:52 (2620:107:4000:c5c2::f3fd:52)  0.546 ms 2620:107:4000:400d:8000:0:6442:3448 (2620:107:4000:400d:8000:0:6442:3448)  0.707 ms 2620:107:4000:4009:8000:0:6442:2658 (2620:107:4000:4009:8000:0:6442:2658)  8.032 ms
 4  2620:107:4000:ad11::f000:e43c (2620:107:4000:ad11::f000:e43c)  0.727 ms 2620:107:4000:400d:8000:0:6442:3626 (2620:107:4000:400d:8000:0:6442:3626)  1.367 ms 2620:107:4000:ad10::f000:e417 (2620:107:4000:ad10::f000:e417)  0.683 ms
 5  2620:107:4000:cfff::f3ff:521 (2620:107:4000:cfff::f3ff:521)  0.658 ms 2620:107:4000:c5c0::f3fd:d (26

In [16]:
traceroute6("google.com",iface='eth0',verbose=True)


Begin emission:
Finished sending 30 packets.

Received 28 packets, got 18 answers, remaining 12 packets
   2607:f8b0:4004:c06::66                    :tcphttp 
4  2620:107:4000:ad10::f000:e419              3       
5  2620:107:4000:cfff::f3ff:1d79              3       
6  2620:107:4000:a710::f000:301b              3       
7  2620:107:4000:cfff::f200:bab1              3       
8  2620:107:4000:cfff::f200:9b21              3       
9  2620:107:4000:2000::d5                     3       
10 2620:107:4000:8001::22                     3       
11 2001:4860:0:1::1b5b                        3       
12 2001:4860:0:1::1070                        3       
13 2001:4860::c:4000:d387                     3       
14 2001:4860::c:4002:6e12                     3       
15 2001:4860::c:4001:9fae                     3       
16 2001:4860::c:4002:1ea3                     3       
17 2001:4860::cc:4002:1f2c                    3       
27 2607:f8b0:4004:c06::66                     SA      
28 2607:f8b0:400

(<Traceroute: TCP:4 UDP:0 ICMP:0 Other:14>,
 <Unanswered: TCP:12 UDP:0 ICMP:0 Other:0>)

In [25]:
traceroute6("facebook.com",iface='eth0',verbose=True)


Begin emission:
Finished sending 30 packets.

Received 37 packets, got 21 answers, remaining 9 packets
   2a03:2880:f103:181:face:b00c:0:25de       :tcphttp 
4  2620:107:4000:ad10::f000:e417              3       
5  2620:107:4000:c5c0::f3fd:e                 3       
6  2620:107:4000:a5d0::f000:2014              3       
7  2620:107:4000:a710::f000:3012              3       
8  2620:107:4000:cfff::f200:9a31              3       
9  2620:107:4000:2000::dd                     3       
10 2620:107:4008:530::2                       3       
11 2620:107:4008:531::2                       3       
12 2620:0:1cff:dead:bef0::14d                 3       
14 2a03:2880:f103:181:face:b00c:0:25de        SA      
15 2a03:2880:f103:181:face:b00c:0:25de        SA      
16 2a03:2880:f103:181:face:b00c:0:25de        SA      
17 2a03:2880:f103:181:face:b00c:0:25de        SA      
18 2a03:2880:f103:181:face:b00c:0:25de        SA      
19 2a03:2880:f103:181:face:b00c:0:25de        SA      
20 2a03:2880:f103

(<Traceroute: TCP:12 UDP:0 ICMP:0 Other:9>,
 <Unanswered: TCP:9 UDP:0 ICMP:0 Other:0>)

In [26]:
ans,uans = traceroute6("ipv6.waybarrios.com",iface='eth0',verbose=True,maxttl=20)

Begin emission:
Finished sending 20 packets.

Received 32 packets, got 17 answers, remaining 3 packets
   2606:4700:3035::ac43:9fea                 :tcphttp 
4  2620:107:4000:ad11::f000:e438              3       
5  2620:107:4000:c5c0::f3fd:19                3       
6  2620:107:4000:c5c0::f3fd:14                3       
7  2620:107:4000:a5d0::f000:201f              3       
8  2620:107:4000:cfff::f200:baa1              3       
9  2620:107:4000:8001::225                    3       
10 2620:107:4008:105::2                       3       
11 2400:cb00:16:3::                           3       
12 2606:4700:3035::ac43:9fea                  SA      
13 2606:4700:3035::ac43:9fea                  SA      
14 2606:4700:3035::ac43:9fea                  SA      
15 2606:4700:3035::ac43:9fea                  SA      
16 2606:4700:3035::ac43:9fea                  SA      
17 2606:4700:3035::ac43:9fea                  SA      
18 2606:4700:3035::ac43:9fea                  SA      
19 2606:4700:3035

#### Packets from Traceroute

In [53]:
#query type
ans[0].query

<IPv6  version=6 tc=0 fl=682892 plen=68 nh=ICMPv6 hlim=61 src=2620:107:4000:ad11::f000:e438 dst=2600:1f18:432d:43a1:6454:f46:6121:cfa3 |<ICMPv6TimeExceeded  type=Time exceeded code=hop limit exceeded in transit cksum=0x3dcd length=0 unused=0x0 |<IPerror6  version=6 tc=0 fl=0 plen=20 nh=TCP hlim=1 src=2600:1f18:432d:43a1:6454:f46:6121:cfa3 dst=2606:4700:3035::ac43:9fea |<TCPerror  sport=12746 dport=http seq=869275962 ack=0 dataofs=5 reserved=0 flags=S window=8192 chksum=0xb70e urgptr=0 |>>>>

In [54]:
#answer type
ans[0].answer

<IPv6  version=6 tc=0 fl=682892 plen=68 nh=ICMPv6 hlim=61 src=2620:107:4000:ad11::f000:e438 dst=2600:1f18:432d:43a1:6454:f46:6121:cfa3 |<ICMPv6TimeExceeded  type=Time exceeded code=hop limit exceeded in transit cksum=0x3dcd length=0 unused=0x0 |<IPerror6  version=6 tc=0 fl=0 plen=20 nh=TCP hlim=1 src=2600:1f18:432d:43a1:6454:f46:6121:cfa3 dst=2606:4700:3035::ac43:9fea |<TCPerror  sport=12746 dport=http seq=869275962 ack=0 dataofs=5 reserved=0 flags=S window=8192 chksum=0xb70e urgptr=0 |>>>>

## Exercise 3


In [144]:
!dig ipv6.waybarrios.com A ipv6.waybarrios.com AAAA



; <<>> DiG 9.18.1-1ubuntu1.1-Ubuntu <<>> ipv6.waybarrios.com A ipv6.waybarrios.com AAAA
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 50276
;; flags: qr rd ra; QUERY: 1, ANSWER: 2, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 65494
;; QUESTION SECTION:
;ipv6.waybarrios.com.		IN	A

;; ANSWER SECTION:
ipv6.waybarrios.com.	250	IN	A	172.67.159.234
ipv6.waybarrios.com.	250	IN	A	104.21.66.126

;; Query time: 0 msec
;; SERVER: 127.0.0.53#53(127.0.0.53) (UDP)
;; WHEN: Fri Aug 26 16:37:20 UTC 2022
;; MSG SIZE  rcvd: 80

;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 17567
;; flags: qr rd ra; QUERY: 1, ANSWER: 2, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 65494
;; QUESTION SECTION:
;ipv6.waybarrios.com.		IN	AAAA

;; ANSWER SECTION:
ipv6.waybarrios.com.	250	IN	AAAA	2606:4700:3035::ac43:9fea
ipv6.waybarrios.com.	250	IN	AAAA	2606:4700:3031::6815:427e

;; 

In [125]:
!dig ipv6.waybarrios.com


; <<>> DiG 9.18.1-1ubuntu1.1-Ubuntu <<>> ipv6.waybarrios.com
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 6900
;; flags: qr rd ra; QUERY: 1, ANSWER: 2, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 65494
;; QUESTION SECTION:
;ipv6.waybarrios.com.		IN	A

;; ANSWER SECTION:
ipv6.waybarrios.com.	285	IN	A	104.21.66.126
ipv6.waybarrios.com.	285	IN	A	172.67.159.234

;; Query time: 0 msec
;; SERVER: 127.0.0.53#53(127.0.0.53) (UDP)
;; WHEN: Fri Aug 26 16:15:49 UTC 2022
;; MSG SIZE  rcvd: 80



In [139]:
#IPv6 version
PUBLIC_DNS = "2606:4700:4700::1111"
ans = sr1(IPv6(dst=PUBLIC_DNS)/UDP()/DNS(qd=DNSQR(qname="ipv6.waybarrios.com", qtype="AAAA"))) 


Begin emission:
Finished sending 1 packets.
.*
Received 2 packets, got 1 answers, remaining 0 packets


In [140]:
ans.show()

###[ IPv6 ]### 
  version   = 6
  tc        = 0
  fl        = 805421
  plen      = 101
  nh        = UDP
  hlim      = 51
  src       = 2606:4700:4700::1111
  dst       = 2600:1f18:432d:43a1:6454:f46:6121:cfa3
###[ UDP ]### 
     sport     = domain
     dport     = domain
     len       = 101
     chksum    = 0xd857
###[ DNS ]### 
        id        = 0
        qr        = 1
        opcode    = QUERY
        aa        = 0
        tc        = 0
        rd        = 1
        ra        = 1
        z         = 0
        ad        = 0
        cd        = 0
        rcode     = ok
        qdcount   = 1
        ancount   = 2
        nscount   = 0
        arcount   = 0
        \qd        \
         |###[ DNS Question Record ]### 
         |  qname     = 'ipv6.waybarrios.com.'
         |  qtype     = AAAA
         |  qclass    = IN
        \an        \
         |###[ DNS Resource Record ]### 
         |  rrname    = 'ipv6.waybarrios.com.'
         |  type      = AAAA
         |  rclass    = IN
  

In [141]:
PUBLIC_DNS_ipv4 = "1.1.1.1"
ans_ipv4 = sr1(IP(dst=PUBLIC_DNS_ipv4)/UDP()/DNS(qd=DNSQR(qname="ipv6.waybarrios.com", qtype="A"))) 

Begin emission:
Finished sending 1 packets.
..*
Received 3 packets, got 1 answers, remaining 0 packets


In [142]:
ans_ipv4.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 97
  id        = 7853
  flags     = DF
  frag      = 0
  ttl       = 52
  proto     = udp
  chksum    = 0x5b06
  src       = 1.1.1.1
  dst       = 172.31.30.184
  \options   \
###[ UDP ]### 
     sport     = domain
     dport     = domain
     len       = 77
     chksum    = 0x1d47
###[ DNS ]### 
        id        = 0
        qr        = 1
        opcode    = QUERY
        aa        = 0
        tc        = 0
        rd        = 1
        ra        = 1
        z         = 0
        ad        = 0
        cd        = 0
        rcode     = ok
        qdcount   = 1
        ancount   = 2
        nscount   = 0
        arcount   = 0
        \qd        \
         |###[ DNS Question Record ]### 
         |  qname     = 'ipv6.waybarrios.com.'
         |  qtype     = A
         |  qclass    = IN
        \an        \
         |###[ DNS Resource Record ]### 
         |  rrname    = 'ipv6.waybarrios.com.'
         |  type  

## Exercise 4

In [143]:
!ping -4 ipv6.waybarrios.com -c 3

PING  (104.21.66.126) 56(84) bytes of data.
64 bytes from 104.21.66.126 (104.21.66.126): icmp_seq=1 ttl=49 time=0.829 ms
64 bytes from 104.21.66.126 (104.21.66.126): icmp_seq=2 ttl=49 time=1.19 ms
64 bytes from 104.21.66.126 (104.21.66.126): icmp_seq=3 ttl=49 time=0.853 ms

---  ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2004ms
rtt min/avg/max/mdev = 0.829/0.956/1.187/0.163 ms


In [152]:
#only ipv4
!dig +noall +answer -t A ipv6.waybarrios.com



ipv6.waybarrios.com.	123	IN	A	104.21.66.126
ipv6.waybarrios.com.	123	IN	A	172.67.159.234


In [154]:
IPv4_host = "104.21.66.126"

In [180]:
long_txt = b"Internet Protocol version 4 is the fourth version of the Internet Protocol.IPv4 uses a 32bit address space which provides 4294967296 unique addresses.34345345345345345345345345345345345345345345345"

In [181]:
len(long_txt)

197

In [177]:
type(long_txt)

bytes

In [182]:
#send long string
send(IP(dst=IPv4_host ) / ICMP() /long_txt , iface='eth0')

.
Sent 1 packets.


#### Sniffer host output - DNS IPv4
```bash
>>> pkts=sniff(iface="eth0",filter="icmp")
>>> pkts.show()
0000 Ether / IP / ICMP 172.31.30.184 > 104.21.66.126 echo-request 0 / Raw
0001 Ether / IP / ICMP 104.21.66.126 > 172.31.30.184 echo-reply 0 / Raw
>>> pkts[0].show()
###[ Ethernet ]###
  dst       = 0a:54:50:8c:3c:29
  src       = 0a:16:23:17:11:01
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 178
     id        = 1
     flags     =
     frag      = 0
     ttl       = 64
     proto     = icmp
     chksum    = 0x4e0
     src       = 172.31.30.184
     dst       = 104.21.66.126
     \options   \
###[ ICMP ]###
        type      = echo-request
        code      = 0
        chksum    = 0xe7b5
        id        = 0x0
        seq       = 0x0
        unused    = ''
###[ Raw ]###
           load      = 'Internet Protocol version 4 is the fourth version of the Internet Protocol.IPv4 uses a 32bit address space which provides 4294967296 unique addresses.'

>>> pkts[1].show()
###[ Ethernet ]###
  dst       = 0a:16:23:17:11:01
  src       = 0a:54:50:8c:3c:29
  type      = IPv4
###[ IP ]###
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 178
     id        = 26034
     flags     =
     frag      = 0
     ttl       = 48
     proto     = icmp
     chksum    = 0xaf2e
     src       = 104.21.66.126
     dst       = 172.31.30.184
     \options   \
###[ ICMP ]###
        type      = echo-reply
        code      = 0
        chksum    = 0xefb5
        id        = 0x0
        seq       = 0x0
        unused    = ''
###[ Raw ]###
           load      = 'Internet Protocol version 4 is the fourth version of the Internet Protocol.IPv4 uses a 32bit address space which provides 4294967296 unique addresses.'
```